In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage,AIMessage
import requests

In [10]:
# Model setup 
import os 
from dotenv import load_dotenv

chat_model_name = os.environ.get("GOOGLE_CHAT_MODEL")

chat_model = ChatGoogleGenerativeAI(model = chat_model_name)

print("Chat model setup successfull...")

Chat model setup successfull...


# create tool 

In [ ]:


@tool
def mul( a : int, b : int) -> int: 
    """Multiply two numbers"""
    return a*b

In [5]:
print(mul.invoke({'a':3,'b':3}))

9


# Tool Binding

In [ ]:
# Tool Binding
llm_with_tools = chat_model.bind_tools([mul])

# Tool Calling

In [13]:
llm_with_tools.invoke("Hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b9947f03-fbd1-4c5e-b253-9b2a7539b140-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}})

In [ ]:
llm_with_tools.invoke("Multiply 3 * 10").tool

AIMessage(content='', additional_kwargs={'function_call': {'name': 'mul', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f00625d4-ea28-4658-b709-02d9060f4a0f-0', tool_calls=[{'name': 'mul', 'args': {'a': 3.0, 'b': 10.0}, 'id': '4639fe05-7668-452a-a9e7-1e69346f2759', 'type': 'tool_call'}], usage_metadata={'input_tokens': 18, 'output_tokens': 5, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}})

In [18]:
llm_with_tools.invoke("Multiply 3 * 10").tool_calls[0]

{'name': 'mul',
 'args': {'a': 3.0, 'b': 10.0},
 'id': '82e16c3d-7fc6-4701-84e4-cd2f8a0f8858',
 'type': 'tool_call'}

# Tool Execution

In [22]:
call_output = llm_with_tools.invoke("Multiply 3 * 10")

In [25]:
call_output.tool_calls[0]

{'name': 'mul',
 'args': {'a': 3.0, 'b': 10.0},
 'id': '287a1196-7551-49b4-9e89-105611bfab4a',
 'type': 'tool_call'}

In [ ]:
mul.invoke({'a': 3.0, 'b': 10.0}) # Only with arguments

30

In [ ]:
# With all all the information

mul.invoke({'name': 'mul',
 'args': {'a': 3.0, 'b': 10.0},
 'id': '287a1196-7551-49b4-9e89-105611bfab4a',
 'type': 'tool_call'})

ToolMessage(content='30', name='mul', tool_call_id='287a1196-7551-49b4-9e89-105611bfab4a')

In [27]:
mul.invoke(call_output.tool_calls[0])

ToolMessage(content='30', name='mul', tool_call_id='287a1196-7551-49b4-9e89-105611bfab4a')

# Workflow

In [36]:
from langchain_core.messages import HumanMessage,AIMessage,ToolMessage


In [44]:
# Step 1: Human Message
query = HumanMessage("Can you multiply 3 with 20")

messages = [query]

print(f"Initial Message: {messages}")

# Step 2: First LLM call to get the tool

response = llm_with_tools.invoke(messages)

messages.append(response)

print(f"LLM Response Tool Call : {response}")

# Step 3: Execute the tool 
# tool_call = response.tool_calls[0]

# tool_output = mul.invoke(tool_call['args'])

# print(f"Message After Tool Output: {tool_output}")

# messages.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call['id']))

tool_output = mul.invoke(response.tool_calls[0])

messages.append(tool_output)

print(f"Message After Tool Output: {tool_output}")

# Step 4: Send the tool's output back to the model

final_output = llm_with_tools.invoke(messages)

print(f"Final Response : {final_output.content}")

Initial Message: [HumanMessage(content='Can you multiply 3 with 20', additional_kwargs={}, response_metadata={})]
LLM Response Tool Call : content='' additional_kwargs={'function_call': {'name': 'mul', 'arguments': '{"a": 3.0, "b": 20.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--d61d0996-3cbc-4579-b858-4a0c2385edd3-0' tool_calls=[{'name': 'mul', 'args': {'a': 3.0, 'b': 20.0}, 'id': '4719d4f8-9030-4ff8-9fda-45c5aa57c9db', 'type': 'tool_call'}] usage_metadata={'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}
Message After Tool Output: content='60' name='mul' tool_call_id='4719d4f8-9030-4ff8-9fda-45c5aa57c9db'
Final Response : The result of multiplying 3 with 20 is 60.
